In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
data = {
    "latitude": [40.7, 34.0, 51.5, 48.8, 35.7],
    "longitude": [-74.0, -118.2, -0.1, 2.3, 139.7],
    "weather_description": ["clear sky", "rain", "snow", "cloudy", "sunny"],
    "temperature": [10, 15, -5, 8, 22],
    "temperature_feels_like": [8, 13, -10, 6, 21],
    "location": ["New York", "Los Angeles", "London", "Paris", "Tokyo"],
    "turn_on_heating": [1, 0, 1, 1, 0],
    "recommended_temperature": [22, 18, 25, 21, 17],
    "extra_items": ["jacket", "umbrella", "heavy coat", "scarf", "none"]
}

In [3]:
# DataFrame
df = pd.DataFrame(data)

In [4]:
df

,latitude,longitude,weather_description,temperature,temperature_feels_like,location,turn_on_heating,recommended_temperature,extra_items
0,40.7,-74.0,clear sky,10,8,New York,1,22,jacket
1,34.0,-118.2,rain,15,13,Los Angeles,0,18,umbrella
2,51.5,-0.1,snow,-5,-10,London,1,25,heavy coat
3,48.8,2.3,cloudy,8,6,Paris,1,21,scarf
4,35.7,139.7,sunny,22,21,Tokyo,0,17,none


In [5]:
# Encoding categorical variables
label_encoder = LabelEncoder()
df["weather_encoded"] = label_encoder.fit_transform(df["weather_description"])
df["location_encoded"] = label_encoder.fit_transform(df["location"])
df["extra_items_encoded"] = label_encoder.fit_transform(df["extra_items"])

In [6]:
# Features and Targets
X = df[["latitude", "longitude", "weather_encoded", "temperature", "temperature_feels_like", "location_encoded"]]
y_heating = df["turn_on_heating"]
y_temp = df["recommended_temperature"]
y_items = df["extra_items_encoded"]

In [7]:
# Train-test split
X_train, X_test, y_heating_train, y_heating_test = train_test_split(X, y_heating, test_size=0.2, random_state=42)
X_train_temp, X_test_temp, y_temp_train, y_temp_test = train_test_split(X, y_temp, test_size=0.2, random_state=42)
X_train_items, X_test_items, y_items_train, y_items_test = train_test_split(X, y_items, test_size=0.2, random_state=42)

In [8]:
# Models
clf_heating = RandomForestClassifier(n_estimators=100, random_state=42)
clf_temp = RandomForestRegressor(n_estimators=100, random_state=42)
clf_items = RandomForestClassifier(n_estimators=100, random_state=42)

In [9]:
# Training models
clf_heating.fit(X_train, y_heating_train)
clf_temp.fit(X_train_temp, y_temp_train)
clf_items.fit(X_train_items, y_items_train)

RandomForestClassifier(random_state=42)

In [12]:
# Recommendation Function
def recommend_settings(latitude, longitude, weather_description, temperature, temperature_feels_like, location):
    if weather_description not in label_encoder.classes_:
        label_encoder.classes_ = np.append(label_encoder.classes_, weather_description)
        weather_enc = label_encoder.transform([weather_description])[0]

    if location not in label_encoder.classes_:
        label_encoder.classes_ = np.append(label_encoder.classes_, location)
        location_enc = label_encoder.transform([location])[0]
    
    features = np.array([[latitude, longitude, weather_enc, temperature, temperature_feels_like, location_enc]])
    heating = clf_heating.predict(features)[0]
    temp_setting = clf_temp.predict(features)[0]
    extra_item = label_encoder.inverse_transform([clf_items.predict(features)[0]])[0]
    
    return {
        "Turn on Heating": bool(heating),
        "Recommended Temperature": round(temp_setting, 1),
        "Extra Item to Carry": extra_item
    }

In [13]:
# Recommendation Results
result = recommend_settings(40.7, -74.0, "clear sky", 5, 2, "New York")
print(result)

{'Turn on Heating': True, 'Recommended Temperature': 20.9, 'Extra Item to Carry': 'scarf'}


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
